**Consider the following model:**

In [ ]:
class ToyModel(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 10, bias=False)
        self.ln = nn.LayerNorm(10)
        self.fc2 = nn.Linear(10, out_features, bias=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.ln(x)
        x = self.fc2(x)
    return x


**Suppose we are training the model on a GPU and that the model parameters are originally in
FP32. We’d like to use autocasting mixed precision with FP16. What are the data types of:**
- the model parameters within the autocast context,
  - FP32
- the output of the first feed-forward layer (ToyModel.fc1),
  - FP16
- the output of layer norm (ToyModel.ln),
  - FP32
- the model’s predicted logits,
  - FP16
- the loss,
  - FP32
- and the model’s gradients?
  - FP32

**You should have seen that FP16 mixed precision autocasting treats the layer normalization layer
differently than the feed-forward layers. What parts of layer normalization are sensitive to mixed
precision? If we use BF16 instead of FP16, do we still need to treat layer normalization differently?
Why or why not?**

Layer Normalization involves two key steps that are prone to numerical instability when performed in $\text{FP16}$ (half-precision):

1. Computing the Variance ($\sigma^2$)

2. Division by Standard Deviation (Square Root and $\epsilon$)

Generally, it is often not necessary to force $\text{LayerNorm}$ into $\text{FP32}$ when using $\text{BF16}$ autocasting.

1. Wider Dynamic Range ($\text{BF16}$): $\text{BF16}$ uses 8 exponent bits, the same number as $\text{FP32}$. This gives it a significantly larger range than $\text{FP16}$ (with its 5 exponent bits). This wider range almost completely eliminates the risk of overflow during the variance summation step.

2. Less Underflow ($\text{BF16}$): Because the exponent range is much wider, $\text{BF16}$ can represent much smaller non-zero numbers than $\text{FP16}$, making it far less prone to underflow to zero in the denominator of the normalization step.

By matching the wide dynamic range of $\text{FP32}$, $\text{BF16}$ offers sufficient numerical stability for operations like $\text{LayerNorm}$, which is why modern frameworks like $\text{PyTorch}$'s Automatic Mixed Precision ($\text{AMP}$) typically permit $\text{LayerNorm}$ to run in $\text{BF16}$ without issue, whereas they force it into $\text{FP32}$ when using $\text{FP16}$.


**Modify your benchmarking script to optionally run the model using mixed precision with BF16.
Time the forward and backward passes with and without mixed-precision for each language model
size described in §1.1.2. Compare the results of using full vs. mixed precision, and comment on
any trends as model size changes. You may find the nullcontext no-op context manager to be
useful.**

| Model   |   Forward Time Avg |   Forward Time Std |   Backward Time Avg |   Backward Time Std | Mixed precision   |
|:--------|-------------------:|-------------------:|--------------------:|--------------------:|:------------------|
| small   |          0.0258554 |        0.00102104  |           0.0508098 |         0.0593376   | False             |
| medium  |          0.049743  |        0.000642239 |           0.0799599 |         0.000597145 | False             |
| large   |          0.106425  |        0.013631    |           0.17753   |         0.000627512 | False             |
| xl      |          0.207034  |        0.0192236   |           0.415999  |         0.196872    | False             |
| 2.7B    |          0.295486  |        0.0420181   |           0.544711  |         0.137999    | False             |
| small   |          0.0464655 |        0.0564612   |           0.0333778 |         0.00205094  | True              |
| medium  |          0.0561096 |        0.003317    |           0.0651349 |         0.00618219  | True              |
| large   |          0.0824193 |        0.00456781  |           0.0927203 |         0.00285367  | True              |
| xl      |          0.109796  |        0.0117926   |           0.150115  |         0.00178366  | True              |
| 2.7B    |          0.106499  |        0.0437887   |           0.227581  |         0.00344241  | True              |
